In [2]:

import h2o 
from h2o.automl import H2OAutoML 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, OrdinalEncoder

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 mins 21 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_erlen_dsuu32
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.312 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [ ]:
path_train = ["hdfs://192.168.1.10/user/data/data_train.csv"]
path_test = ["hdfs://192.168.1.10/user/data/data_test.csv"]
train = h2o.import_file(path=path_train)
test  = h2o.import_file(path=path_test)

In [ ]:
h2o.init()

X = pd.read_csv("data/stores_train_new.csv").drop(["district_name", "mall_bool", "municipality_name", "all_households"], axis=1)
# X2 = pd.read_csv("data/stores_train_new.csv").drop(["district_name", "mall_bool", "municipality_name", "all_households"], axis=1)



X["is_grocery"] = np.where(X["lv4_desc"]=="Grocery stores",1,0)

# X.loc[X['lv4_desc'] =="Grocery stores"]

X.head()

X = pd.read_csv("data/stores_train_new2.csv").drop(["district_name", "mall_name", "municipality_name", "lv1_desc", "lv2_desc", "lv3_desc", "interval1", "interval5"], axis=1)
X_test = pd.read_csv("data/stores_test_new.csv").drop(["district_name", "mall_name", "municipality_name", "lv1_desc", "lv2_desc", "lv3_desc", "interval1", "interval5"], axis=1)


# transformer_y = StandardScaler().fit(y_train)


X = X.loc[X["revenue"]!= 0] #fjerner de som har 0 i revenue


X["revenue"] = np.log10(X["revenue"])

h2o_frame = h2o.H2OFrame(X)

x_train = h2o_frame.columns
y_train = "revenue"
x_train = x_train.remove(y_train)

model = H2OAutoML(sort_metric='RMSLE', max_models=10, exclude_algos=["DeepLearning"], seed=666)

model.train(x=x_train, y=y_train, training_frame=h2o_frame)

X_test = h2o.H2OFrame(X_test)

y_pred = model.predict(X_test)



y_pred = h2o.as_list(y_pred)


orig_test_data = pd.read_csv('data/stores_test.csv')



y_pred = 10 ** y_pred


print(y_pred[y_pred<0].shape)
y_pred[y_pred<0] = 0


submission = pd.DataFrame()
submission['id'] = orig_test_data.store_id
submission['predicted'] = y_pred

filepath_test = Path('sample_submission.csv')

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv(filepath_test, index=False)
submission